In [4]:
import tensorflow as tf
import numpy as np

def fun_range(fun_index):
    d = 30
    if fun_index == 1:
        l = [-100]
        u = [100]
    elif fun_index == 2:
        l = [-30]
        u = [30]
    elif fun_index == 3:
        l = [-500]
        u = [500]
    elif fun_index == 4:
        l = [-512]
        u = [512]
        d = 2
    elif fun_index == 5:
        l = [-5, 0]
        u = [10, 15]
        d = 2
    elif fun_index == 6:
        l = [-50]
        u = [50]
    return l, u, d


def ben_function(x, function_index):
    # Sphere
    if function_index == 1:
        s = sum(np.square(x))
    # Rosenbrock
    elif function_index == 2:
        s = sum(100.0 * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1 - x[:-1]) ** 2.0)
    # Schwefel
    elif function_index == 3:
        s = sum(x * np.sin(np.sqrt(abs(x))))
    # Eggholder
    elif function_index == 4:
        s = -(x[1] + 47.0) * np.sin(np.sqrt(abs(x[0] / 2 + (x[1] + 47.0)))) - x[0] * np.sin(
            np.sqrt(abs(x[0] - (x[1] + 47.0))))
    # Branin
    elif function_index == 5:
        s = ((x[1] - 5.1 / (4 * np.pi ** 2) * x[0] ** 2 + 5 / np.pi * x[0] - 6) ** 2 + 10 * (
                1 - 1 / (8 * np.pi)) * np.cos(x[0]) + 10)
    # Penalized
    elif function_index == 6:
        a = 10
        k = 100
        m = 4
        Dim = len(x)
        s = (np.pi / Dim) * (10 * ((np.sin(np.pi * (1 + (x[0] + 1) / 4))) ** 2) + sum(
            (((x[:-1] + 1) / 4) ** 2) * (1 + 10 * ((np.sin(np.pi * (1 + (x[1:] + 1) / 4)))) ** 2)) + (
                                  (x[Dim - 1] + 1) / 4) ** 2) + sum(
            k * ((x - a) ** m) * (x > a) + k * ((-x - a) ** m) * (x < (-a)))
    return s
def change_dir():
    global aha
    aha = tf.keras.optimizers.experimental.Adagrad(learning_rate=0.001)


class AHAOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, fun_index, max_it, npop, learning_rate=0.001, **kwargs):
        self.fun_index = fun_index
        self.max_it = max_it
        self.npop = npop
        self.learning_rate = learning_rate
        kwargs["name"] = "AHA"
        super(AHAOptimizer, self).__init__(**kwargs)

    def lb_ub_dim(self):
        lb, ub, dim = fun_range(self.fun_index)
        if len(lb) == 1:
            lb = lb * dim
            ub = ub * dim
        return lb, ub, dim

    def ben_functions(self, x):
        return ben_function(x, self.fun_index)

    def space_bound(self, X, Up, Low):
        dim = len(X)
        S = (X > Up) + (X < Low)
        res = (np.random.rand(dim) * (np.array(Up) - np.array(Low)) + np.array(Low)) * S + X * (~S)
        return res

    def AHA_optimize(self, variables):
        lb, ub, dim = self.lb_ub_dim()
        pop_pos = np.zeros((self.npop, dim))
        for i in range(dim):
            pop_pos[:, i] = np.random.uniform(lb[i], ub[i], self.npop)
        pop_cost = np.zeros(self.npop)
        for i in range(self.npop):
            pop_cost[i] = self.ben_functions(pop_pos[i, :])
        for it in range(self.max_it):
            for i in range(self.npop):
                for j in range(self.npop):
                    if i != j:
                        temp = pop_pos[i, :] - pop_pos[j, :]
                        temp = self.space_bound(temp, 2 * np.abs(ub - lb), -2 * np.abs(ub - lb))
                        pop_pos[i, :] = pop_pos[i, :] + temp * np.random.uniform(0, 1, 1) * self.learning_rate
                        pop_cost[i] = self.ben_functions(pop_pos[i, :])
        best_pos = pop_pos[np.argmin(pop_cost), :]
        return best_pos
# This function will update the weights of the neural network .
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        variables = [p.numpy() for p in params]
        best_pos = self.AHA_optimize(variables)
        updates = [tf.assign(p, v) for p, v in zip(params, best_pos)]

# Define AHA optimizer
aha = AHAOptimizer(fun_index=1, max_it=100, npop=50, learning_rate=0.001)



import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
change_dir()

# Load data from CSV file
train_data = pd.read_csv('train.csv' , sep=";")
test_data = pd.read_csv('test.csv' , sep=";")

#Checking for Null values
print("Checking for Null values")
print(train_data.isnull().sum())

# Filling Null values
data = train_data.fillna(train_data.mean())

#Checking for Null values after filling
print("Checking for Null values after filling")
print(train_data.isnull().sum())

# Checking for Duplicates
print("Checking for Duplicates")
print(data.duplicated().sum())

# Removing Duplicates
data = data.drop_duplicates()

# Checking for Duplicates after removing
print("Checking for Duplicates after removing")
print(data.duplicated().sum())

X_train = train_data.iloc[:, :-1] # Features
y_train = train_data.iloc[:, -1] # Target
X_test = test_data.iloc[:, :-1] # Features
y_test = test_data.iloc[:, -1] # Target


# Scale the data (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=aha, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=32)

# Make predictions
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5) # Convert probabilities to binary class labels

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Checking for Null values
profile pic             0
nums/length username    0
fullname words          0
nums/length fullname    0
name==username          0
description length      0
external URL            0
private                 0
#posts                  0
#followers              0
#follows                0
fake                    0
dtype: int64
Checking for Null values after filling
profile pic             0
nums/length username    0
fullname words          0
nums/length fullname    0
name==username          0
description length      0
external URL            0
private                 0
#posts                  0
#followers              0
#follows                0
fake                    0
dtype: int64
Checking for Duplicates
2
Checking for Duplicates after removing
0
Epoch 1/200
18/18 [==============================] - 1s 5ms/step - loss: 0.6840 - accuracy: 0.5000
Epoch 2/200
18/18 [==============================] - 0s 5ms/step - loss: 0.6832 - accuracy: 0.5069
Epoch 3/200
18/18 [==